In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

from numpy.polynomial.polynomial import polyfit

import pyarrow as pa
import pyarrow.parquet as pq

import statsmodels.api as sm
import statsmodels
#import statsmodels.formula.api as smf
from linearmodels.iv import IV2SLS
from linearmodels.panel import PanelOLS

import warnings
warnings.filterwarnings("ignore")

C:\ProgramData\Anaconda3\lib\site-packages\linearmodels\panel\data.py:10: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import (Categorical, DataFrame, Index, MultiIndex, Panel, Series,


### First Steps

Here we are going to combine the trade and autos data set...

In [2]:
fig_path = "C:\\github\\expenditure_tradeshocks\\figures"

In [3]:
file_path = os.getcwd()
#file_path = "F:\\POLK AUTO DATA\\"

#trade_county = pq.read_table(file_path + "\\data\\trade_employment_all.parquet").to_pandas()

trade_county = pq.read_table(file_path + "\\data\\trade_employment_2014.parquet").to_pandas()

In [4]:
trade_county["time"] = pd.to_datetime(trade_county.time)

trade_county.set_index(["area_fips", "time"],inplace = True)

In [5]:
trade_county.head(30)

total_exp_pc  china_exp_pc    tariff  emplvl_2017  \
area_fips time                                                            
10001     2014-01-01           NaN           NaN  1.069526          NaN   
          2014-02-01           NaN           NaN  1.069526          NaN   
          2014-03-01           NaN           NaN  1.069526          NaN   
          2014-04-01           NaN           NaN  1.069526          NaN   
          2014-05-01           NaN           NaN  1.069526          NaN   
          2014-06-01           NaN           NaN  1.069526          NaN   
          2014-07-01           NaN           NaN  1.069526          NaN   
          2014-08-01           NaN           NaN  1.069526          NaN   
          2014-09-01           NaN           NaN  1.069526          NaN   
          2014-10-01           NaN           NaN  1.069526          NaN   
          2014-11-01           NaN           NaN  1.069526          NaN   
          2014-12-01           NaN           NaN  1.069526          NaN   
          2015-01-01    523.035475     57.171736  1.069526       2843.0   
          2015-02-01    505.746088     47.313771  1.069501       2843.0   
          2015-03-01    558.254429     45.962111  1.069501       2843.0   
          2015-04-01    526.297224     38.689461  1.069500       2843.0   
          2015-05-01    511.775455     35.258126  1.069500       2843.0   
          2015-06-01    487.368675     30.555724  1.069500       2843.0   
          2015-07-01    470.778081     33.287052  1.069500       2843.0   
          2015-08-01    472.453582     29.613066  1.069500       2843.0   
          2015-09-01    461.429813     34.061929  1.069499       2843.0   
          2015-10-01    527.824207     76.012207  1.069499       2843.0   
          2015-11-01    516.968534     69.262706  1.069499       2843.0   
          2015-12-01    488.830832     51.260005  1.069499       2843.0   
          2016-01-01    453.257185     47.280196  1.069532       2843.0   
          2016-02-01    471.930726     47.211522  1.069499       2843.0   
          2016-03-01    485.376760     35.078484  1.069500       2843.0   
          2016-04-01    460.259354     27.991526  1.069500       2843.0   
          2016-05-01    473.572638     28.235163  1.069499       2843.0   
          2016-06-01    476.828261     28.236923  1.069499       2843.0   

                       fips  total_employment  emp_all  emp_gds  emp_n_gds  \
area_fips time                                                               
10001     2014-01-01   None           29514.0      0.0      0.0    37089.0   
          2014-02-01   None           29514.0      0.0      0.0    37024.0   
          2014-03-01   None           29514.0      0.0      0.0    37254.0   
          2014-04-01   None           29514.0      0.0      0.0    37875.0   
          2014-05-01   None           29514.0      0.0      0.0    38005.0   
          2014-06-01   None           29514.0      0.0      0.0    38195.0   
          2014-07-01   None           29514.0      0.0      0.0    37797.0   
          2014-08-01   None           29514.0      0.0      0.0    37584.0   
          2014-09-01   None           29514.0      0.0      0.0    37770.0   
          2014-10-01   None           29514.0      0.0      0.0    38032.0   
          2014-11-01   None           29514.0      0.0      0.0    38546.0   
          2014-12-01   None           29514.0      0.0      0.0    38880.0   
          2015-01-01  10001           29514.0      0.0      0.0    37604.0   
          2015-02-01  10001           29514.0      0.0      0.0    37349.0   
          2015-03-01  10001           29514.0      0.0      0.0    37705.0   
          2015-04-01  10001           29514.0      0.0      0.0    38351.0   
          2015-05-01  10001           29514.0      0.0      0.0    38674.0   
          2015-06-01  10001           29514.0      0.0      0.0    38847.0   
          2015-07-01  10001           29514.0      0.0      0

In [6]:
trade_county["log_tariff"] = np.log(1+.01*trade_county["tariff"])

trade_county["log_exp_total"] = np.log(trade_county["total_exp_pc"]).replace(-np.inf, np.nan)

trade_county["log_exp_china"] = np.log(trade_county["china_exp_pc"]).replace(-np.inf, np.nan)

trade_county["log_employment"] = np.log(trade_county["emp_gds"]).replace(-np.inf, np.nan)

trade_county["const"] = 1

In [7]:
trade_county.reset_index(inplace = True)

trade_county.rename({"area_fips": "GEOFIPS"},axis = 1, inplace = True)

trade_county["state_fips"] = trade_county["GEOFIPS"].astype(str).str[0:2]

trade_county["GEOFIPS"] = trade_county["GEOFIPS"].astype(int)

In [8]:
#trade_county["log_value"] = np.log(trade_county["value"]).replace(-np.inf,np.nan)

trade_county.set_index(["GEOFIPS", "time"], inplace = True)

In [9]:
trade_county.head()

total_exp_pc  china_exp_pc    tariff  emplvl_2017  fips  \
GEOFIPS time                                                                  
10001   2014-01-01           NaN           NaN  1.069526          NaN  None   
        2014-02-01           NaN           NaN  1.069526          NaN  None   
        2014-03-01           NaN           NaN  1.069526          NaN  None   
        2014-04-01           NaN           NaN  1.069526          NaN  None   
        2014-05-01           NaN           NaN  1.069526          NaN  None   

                    total_employment  emp_all  emp_gds  emp_n_gds  \
GEOFIPS time                                                        
10001   2014-01-01           29514.0      0.0      0.0    37089.0   
        2014-02-01           29514.0      0.0      0.0    37024.0   
        2014-03-01           29514.0      0.0      0.0    37254.0   
        2014-04-01           29514.0      0.0      0.0    37875.0   
        2014-05-01           29514.0      0.0      0.0    38005.0   

                    rural_share  2010_population  2017_income  \
GEOFIPS time                                                    
10001   2014-01-01     0.269694         162310.0      57647.0   
        2014-02-01     0.269694         162310.0      57647.0   
        2014-03-01     0.269694         162310.0      57647.0   
        2014-04-01     0.269694         162310.0      57647.0   
        2014-05-01     0.269694         162310.0      57647.0   

                    2017_population  log_tariff  log_exp_total  log_exp_china  \
GEOFIPS time                                                                    
10001   2014-01-01         173145.0    0.010638            NaN            NaN   
        2014-02-01         173145.0    0.010638            NaN            NaN   
        2014-03-01         173145.0    0.010638            NaN            NaN   
        2014-04-01         173145.0    0.010638            NaN            NaN   
        2014-05-01         173145.0    0.010638            NaN            NaN   

                    log_employment  const state_fips  
GEOFIPS time                                          
10001   2014-01-01             NaN      1         10  
        2014-02-01             NaN      1         10  
        2014-03-01             NaN      1         10  
        2014-04-01             NaN      1         10  
        2014-05-01             NaN      1         10

In [10]:
# First take growth rates

# note , I'm a bit confused about why the sorting,
# here is that because of some missing values, the resulting dateframe from the 
# first operation may be out of place, so we need to resort things to make sure that
# the time difference is correct.

trade_county["tariff_change"] = trade_county.groupby(["GEOFIPS"]).tariff.diff(12)

trade_county.sort_values(["GEOFIPS", "time"], inplace = True)

trade_county["log_tariff_change"] = trade_county.groupby(["GEOFIPS"]).log_tariff.diff(12)

trade_county.sort_values(["GEOFIPS", "time"], inplace = True)

trade_county["total_trade_growth"] = trade_county.groupby(["GEOFIPS"]).log_exp_total.diff(12)

trade_county.sort_values(["GEOFIPS", "time"], inplace = True)

trade_county["china_trade_growth"] = trade_county.groupby(["GEOFIPS"]).log_exp_china.diff(12)

trade_county.sort_values(["GEOFIPS", "time"], inplace = True)

### Regression Analysis


In [11]:
trade_county.columns

Index(['total_exp_pc', 'china_exp_pc', 'tariff', 'emplvl_2017', 'fips',
       'total_employment', 'emp_all', 'emp_gds', 'emp_n_gds', 'rural_share',
       '2010_population', '2017_income', '2017_population', 'log_tariff',
       'log_exp_total', 'log_exp_china', 'log_employment', 'const',
       'state_fips', 'tariff_change', 'log_tariff_change',
       'total_trade_growth', 'china_trade_growth'],
      dtype='object')

#### Total Trade, Weighted

In [12]:
start = "2018-01-01"

In [13]:
all_vars = ["const", "log_tariff_change", 'total_trade_growth','2017_population']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,start:"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

weights = dataset['2017_population'].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)
# Some countys because of there size and how the QCEW deals with stuff, there are is zero.
# PanelOLS does not like 0 weight, so this the the work around. It does not matter. 


mod = PanelOLS(dataset.total_trade_growth, dataset[exog_vars], weights = weights)

fe_res = mod.fit(cov_type='clustered', cluster_entity=True)

print(fe_res)

15
3252
                          PanelOLS Estimation Summary                           
Dep. Variable:     total_trade_growth   R-squared:                        0.0974
Estimator:                   PanelOLS   R-squared (Between):             -0.0045
No. Observations:               43755   R-squared (Within):               0.1814
Date:                Mon, Nov 18 2019   R-squared (Overall):              0.0974
Time:                        16:49:28   Log-likelihood                 4.475e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                   1.694e+04
Entities:                        2917   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                 F(1,43753)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             457.68
                    

In [14]:
all_vars = ["const", "log_tariff_change", 'total_trade_growth','2017_population']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,start:"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

weights = dataset['2017_population'].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)
# Some countys because of there size and how the QCEW deals with stuff, there are is zero.
# PanelOLS does not like 0 weight, so this the the work around. It does not matter. 


mod = PanelOLS(dataset.total_trade_growth, dataset[exog_vars], weights = weights,
               time_effects = True)

fe_res = mod.fit(cov_type='clustered', cluster_entity=True)

print(fe_res)

15
3252
                          PanelOLS Estimation Summary                           
Dep. Variable:     total_trade_growth   R-squared:                        0.0164
Estimator:                   PanelOLS   R-squared (Between):              0.0194
No. Observations:               43755   R-squared (Within):               0.1035
Date:                Mon, Nov 18 2019   R-squared (Overall):              0.0655
Time:                        16:49:29   Log-likelihood                 4.892e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                   1.551e+04
Entities:                        2917   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                 F(1,43739)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             108.92
                    

In [25]:
all_vars = ["const", "log_tariff_change", 'total_trade_growth','2017_population']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,start:"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

weights = dataset['2017_population'].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)
# Some countys because of there size and how the QCEW deals with stuff, there are is zero.
# PanelOLS does not like 0 weight, so this the the work around. It does not matter. 


mod = PanelOLS(dataset.total_trade_growth, dataset[exog_vars], weights = weights, entity_effects=True,
               time_effects = True)

fe_res = mod.fit(cov_type='clustered', cluster_entity=True)

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:     total_trade_growth   R-squared:                        0.0023
Estimator:                   PanelOLS   R-squared (Between):              0.0044
No. Observations:               44355   R-squared (Within):               0.0269
Date:                Mon, Nov 18 2019   R-squared (Overall):              0.0183
Time:                        16:57:23   Log-likelihood                 2.092e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      102.42
Entities:                        2957   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                 F(1,44339)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             26.060
                            

#### Total Trade, unweighted

In [16]:
all_vars = ["const", "log_tariff_change", 'total_trade_growth','2017_population']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,start:"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

weights = dataset['2017_population'].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)
# Some countys because of there size and how the QCEW deals with stuff, there are is zero.
# PanelOLS does not like 0 weight, so this the the work around. It does not matter. 


mod = PanelOLS(dataset.total_trade_growth, dataset[exog_vars])

fe_res = mod.fit(cov_type='clustered', cluster_entity=True)

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:     total_trade_growth   R-squared:                        0.0459
Estimator:                   PanelOLS   R-squared (Between):             -0.0225
No. Observations:               44355   R-squared (Within):               0.0886
Date:                Mon, Nov 18 2019   R-squared (Overall):              0.0459
Time:                        16:49:30   Log-likelihood                 1.673e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2135.3
Entities:                        2957   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                 F(1,44353)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             207.38
                            

In [17]:
all_vars = ["const", "log_tariff_change", 'total_trade_growth','2017_population']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,start:"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

weights = dataset['2017_population'].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)
# Some countys because of there size and how the QCEW deals with stuff, there are is zero.
# PanelOLS does not like 0 weight, so this the the work around. It does not matter. 

mod = PanelOLS(dataset.total_trade_growth, dataset[exog_vars], time_effects = True)

fe_res = mod.fit(cov_type='clustered', cluster_entity=True)

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:     total_trade_growth   R-squared:                        0.0023
Estimator:                   PanelOLS   R-squared (Between):              0.0044
No. Observations:               44355   R-squared (Within):               0.0269
Date:                Mon, Nov 18 2019   R-squared (Overall):              0.0183
Time:                        16:49:31   Log-likelihood                 2.092e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      102.42
Entities:                        2957   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                 F(1,44339)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             26.060
                            

In [18]:
all_vars = ["const", "log_tariff_change", 'total_trade_growth','2017_population']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,start:"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

weights = dataset['2017_population'].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)
# Some countys because of there size and how the QCEW deals with stuff, there are is zero.
# PanelOLS does not like 0 weight, so this the the work around. It does not matter. 

mod = PanelOLS(dataset.total_trade_growth, dataset[exog_vars], time_effects = True,
                              entity_effects=True)

fe_res = mod.fit(cov_type='clustered', cluster_entity=True)

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:     total_trade_growth   R-squared:                        0.0006
Estimator:                   PanelOLS   R-squared (Between):              0.0032
No. Observations:               44355   R-squared (Within):               0.0160
Date:                Mon, Nov 18 2019   R-squared (Overall):              0.0111
Time:                        16:49:31   Log-likelihood                 3.562e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      25.900
Entities:                        2957   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                 F(1,41383)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             5.9734
                            

#### Chine Trade, Weighted

In [19]:
all_vars = ["const", "log_tariff_change", 'china_trade_growth','2017_population']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,start:"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

weights = dataset['2017_population'].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)

mod = PanelOLS(dataset.china_trade_growth, dataset[exog_vars], weights = weights)

fe_res = mod.fit(cov_type='clustered', cluster_entity = True)

print(fe_res)

15
3252
                          PanelOLS Estimation Summary                           
Dep. Variable:     china_trade_growth   R-squared:                        0.2403
Estimator:                   PanelOLS   R-squared (Between):              0.2186
No. Observations:               43755   R-squared (Within):               0.2490
Date:                Mon, Nov 18 2019   R-squared (Overall):              0.2403
Time:                        16:49:32   Log-likelihood                 -1.66e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                   1.053e+04
Entities:                        2917   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                 F(1,43753)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             743.80
                    

In [20]:
all_vars = ["const", "log_tariff_change", 'china_trade_growth','2017_population']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,start:"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

weights = dataset['2017_population'].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)

mod = PanelOLS(dataset.china_trade_growth, dataset[exog_vars], weights = weights, time_effects = True)

fe_res = mod.fit(cov_type='clustered', cluster_entity = True)

print(fe_res)

15
3252
                          PanelOLS Estimation Summary                           
Dep. Variable:     china_trade_growth   R-squared:                        0.1124
Estimator:                   PanelOLS   R-squared (Between):              0.2243
No. Observations:               43755   R-squared (Within):               0.2175
Date:                Mon, Nov 18 2019   R-squared (Overall):              0.2195
Time:                        16:49:33   Log-likelihood                 -1.36e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1744.0
Entities:                        2917   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                 F(1,43739)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             308.92
                    

In [21]:
all_vars = ["const", "log_tariff_change", 'china_trade_growth','2017_population']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,start:"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

weights = dataset['2017_population'].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)

mod = PanelOLS(dataset.china_trade_growth, dataset[exog_vars], weights = weights, time_effects = True,
               entity_effects=True)

fe_res = mod.fit(cov_type='clustered', cluster_entity = True)

print(fe_res)

15
3252
                          PanelOLS Estimation Summary                           
Dep. Variable:     china_trade_growth   R-squared:                        0.0462
Estimator:                   PanelOLS   R-squared (Between):              0.2027
No. Observations:               43755   R-squared (Within):               0.1856
Date:                Mon, Nov 18 2019   R-squared (Overall):              0.1905
Time:                        16:49:35   Log-likelihood                   -4513.7
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      6316.6
Entities:                        2917   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                 F(1,40823)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             183.20
                    

#### China Trade Unweighted

In [22]:
all_vars = ["const", "log_tariff_change", 'china_trade_growth','2017_population']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,start:"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

weights = dataset['2017_population'].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)

mod = PanelOLS(dataset.china_trade_growth, dataset[exog_vars])

fe_res = mod.fit(cov_type='clustered', cluster_entity = True)

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:     china_trade_growth   R-squared:                        0.1749
Estimator:                   PanelOLS   R-squared (Between):              0.1540
No. Observations:               44355   R-squared (Within):               0.1821
Date:                Mon, Nov 18 2019   R-squared (Overall):              0.1749
Time:                        16:49:35   Log-likelihood                -4.287e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      9399.1
Entities:                        2957   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                 F(1,44353)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             393.88
                            

In [23]:
all_vars = ["const", "log_tariff_change", 'china_trade_growth','2017_population']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,start:"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

weights = dataset['2017_population'].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)

mod = PanelOLS(dataset.china_trade_growth, dataset[exog_vars], time_effects = True)

fe_res = mod.fit(cov_type='clustered', cluster_entity = True)

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:     china_trade_growth   R-squared:                        0.0827
Estimator:                   PanelOLS   R-squared (Between):              0.1707
No. Observations:               44355   R-squared (Within):               0.1489
Date:                Mon, Nov 18 2019   R-squared (Overall):              0.1546
Time:                        16:49:35   Log-likelihood                -3.811e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      3999.8
Entities:                        2957   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                 F(1,44339)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             312.10
                            

In [24]:
all_vars = ["const", "log_tariff_change", 'china_trade_growth','2017_population']

idx = pd.IndexSlice

#weights = trade_county["emplvl_2017"].loc[idx[:,"2018-01-01":"2019-02-01"]]

dataset = trade_county[all_vars].loc[idx[:,start:"2019-06-01"],:]

exog_vars = ["const","log_tariff_change"]

weights = dataset['2017_population'].to_frame()

weights.replace(to_replace = 0, value = 0.001,inplace = True)

mod = PanelOLS(dataset.china_trade_growth, dataset[exog_vars], time_effects = True,
               entity_effects=True)

fe_res = mod.fit(cov_type='clustered', cluster_entity = True)

print(fe_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:     china_trade_growth   R-squared:                        0.0358
Estimator:                   PanelOLS   R-squared (Between):              0.1591
No. Observations:               44355   R-squared (Within):               0.1295
Date:                Mon, Nov 18 2019   R-squared (Overall):              0.1371
Time:                        16:49:36   Log-likelihood                -2.943e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1538.6
Entities:                        2957   P-value                           0.0000
Avg Obs:                       15.000   Distribution:                 F(1,41383)
Min Obs:                       15.000                                           
Max Obs:                       15.000   F-statistic (robust):             239.06
                            